In [1]:
import numpy as np

from src.graphic_interface.simu_result import load_all_simulation
from src.surface.surface_Fourier import Surface_Fourier,plot,plot_function_on_surface

#plt.rcParams['figure.figsize'] = [16, 4]
#plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [2]:
column_labels=["Name","lambda","Penalization","cost B","max B","cost j","max j","EM cost","Distance","perimeter","Max curv","nit"]
path='tmp/'
lst_simu=load_all_simulation(path)
values_lambda = set(map(lambda x:x.lamb, lst_simu))
newlist_simu = [[y for y in lst_simu if y.lamb==x] for x in values_lambda]

without_curv_lambda_small
with_curv_lambda_small
with_curv
output_test


In [3]:
data=[]
lst_surf=[]
for i in range(len(values_lambda)):
    #we add the ref
    partial_lst=newlist_simu[i]
    elt=partial_lst[0]
    data_dic=elt.get_data_dic_ref()
    data.append([data_dic[col_name] for col_name in column_labels])#get the item corresponding to the column label
    lst_surf.append(elt.S_ref)
    for elt in partial_lst:
        data_dic=elt.get_data_dic()
        data.append([data_dic[col_name] for col_name in column_labels])#get the item corresponding to the column label
        lst_surf.append(elt.S)
for i,row in enumerate(data):
    row.append(str(i))

In [4]:
column_labels_latex=["Name","\lambda","Penalization","\chi_B^2","|B_{err}|_\infty","\chi_j^2","|j|_\infty","EM cost","\\text{Distance } (m)","\\text{Perimeter }(m^2)","\\text{Maximal curvature }(m^{-1})","\\text{number of iteration}"," number"]
tex="\\begin{array}{"+'S'*len(column_labels_latex)+'} \n'
tex+= '&'.join(column_labels_latex)+'\\\\ \n'
for row in data:
    tex+= '&'.join(row)+'\\\\ \n'
tex+="\\end{array}"
#print(tex)


In [5]:
from IPython.display import display, Math, Latex
display(Math(tex))

<IPython.core.display.Math object>

In [6]:
Sp_parametrization=Surface_Fourier.load_file('data/li383/plasma_surf.txt')
Sp=Surface_Fourier(Sp_parametrization,(64,64),3)

In [ ]:
To_plot=[Sp,lst_surf[5]]
plot(To_plot)

In [28]:
S=lst_surf[1]
plot_function_on_surface(S,np.maximum(np.abs(S.principles[0]),np.abs(S.principles[1])))


In [7]:
newlist_simu[0][0].result


        2.77366537e-03, -2.45916467e-03, -3.76877002e-03, -7.79232968e-04,
        1.91164886e-03, -2.92878717e-03, -1.51902411e-03, -1.18392000e-03,
        2.78276310e-03,  1.56212766e-03, -5.82837505e-04,  1.16278146e-02,
       -7.89359721e-03, -1.91531582e-02, -1.25418703e-02, -3.38664265e-03,
        8.76371284e-03, -7.68189663e-04, -1.14842990e-02,  3.33720214e-03,
       -4.04199674e-03, -1.39889166e-03, -2.42638610e-03, -1.43162020e-03,
       -1.31078155e-03, -2.92701340e-03, -9.64415228e-04, -3.36830175e-03,
        1.33959716e-04,  5.80814344e-04, -1.77649617e-03, -1.79402362e-03,
        1.04433718e-02,  4.94537659e-04, -2.51522584e-03, -8.96983032e-04,
        1.00080560e-02,  1.05329798e-03, -3.41227948e-03, -2.41802557e-03,
        3.88498133e-05, -3.75390247e-03,  7.32952671e-04,  2.80409670e-03,
       -6.07646437e-05,  1.43105611e-03,  1.58376481e-03, -1.22581706e-04,
        2.04080671e-05, -1.41826645e-03,  2.67329019e-03, -5.45777619e-03,
        4.29908405e-03, 

In [ ]:

        )
    def load_simu(self,event):
        print(self.sim_path._get_text())
        self.lst_simu+=load_all_simulation(self.sim_path._get_text())
        new_lst=[]
        for elt in self.lst_simu:
        #    data_dic=elt.get_data_dic()
            #print(data_dic)
            #print([data_dic[col_name] for col_name in list(zip(*self.data_tables.column_data))[0]])
       #     print('hi')
        new_lst.append([data_dic[col_name] for col_name in list(zip(*self.data_tables.column_data))[0]])#get the item corresponding to the column label
            #self.data_tables.row_data.append(
        #print(list(zip(*self.data_tables.column_data))[0])
        print(new_lst)
        new_lst=
        self.data_tables.row_data=new_lst

List_simu().run()